In [10]:
import numpy as np
import pandas as pd
from sqlalchemy import text
from connection import connect
from deep_translator import GoogleTranslator

In [11]:
_, etl_conn, co_sa = connect()


Esquemas encontrados: ['hr', 'person', 'production', 'public', 'purchasing', 'sales']

--- Tablas de Negocio Encontradas ---
Esquema 'hr' (6 tablas):
  > ['department', 'employee', 'employee_department_history', 'employee_pay_history', 'job_candidate']...
Esquema 'person' (13 tablas):
  > ['business_entity', 'address', 'address_type', 'business_entity_address', 'business_entity_contact']...
Esquema 'production' (25 tablas):
  > ['illustration', 'bill_of_materials', 'culture', 'document', 'location']...
Esquema 'public' (4 tablas):
  > ['awbuild_version', 'database_log', 'error_log', 'sysdiagrams']...
Esquema 'purchasing' (5 tablas):
  > ['product_vendor', 'purchase_order_detail', 'purchase_order_header', 'ship_method', 'vendor']...
Esquema 'sales' (19 tablas):
  > ['country_region_currency', 'credit_card', 'customer', 'currency', 'currency_rate']...


Crear un countryName en español, SpanishCountryName

codigo sql

    SELECT
        a.address_id AS address_alternate_key,
        a.city,
        a.postal_code,
        sp.state_province_code,
        sp.name AS state_province_name,
        sp.country_region_code,
		cr.name,
        NULL AS sales_territory_key,
        a.modified_date
    FROM person.address a
    INNER JOIN person.state_province sp
        ON a.state_province_id = sp.state_province_id
	INNER JOIN person.country_region cr
		ON sp.country_region_code = cr.country_region_code;

Tabla, crear fuera de un schema publico

    CREATE TABLE IF NOT EXISTS dw.dim_geography (
        geography_key SERIAL PRIMARY KEY,
        address_alternate_key INT,
        city TEXT,
        postal_code TEXT,
        state_province_code TEXT,
        state_province_name TEXT,
        country_region_code TEXT,
        spanish_country_region_name TEXT,
        sales_territory_key INT,
        modified_date TIMESTAMP
    );


# Extract

In [12]:
query_geography = text("""
    SELECT
        a.address_id AS address_alternate_key,
        a.city,
        a.postal_code,
        sp.state_province_code,
        sp.name AS state_province_name,
        sp.country_region_code,
		cr.name AS country_name,
        NULL AS sales_territory_key,
        a.modified_date
    FROM person.address a
    INNER JOIN person.state_province sp
        ON a.state_province_id = sp.state_province_id
	INNER JOIN person.country_region cr
		ON sp.country_region_code = cr.country_region_code;
""")

df_geo = pd.read_sql(query_geography, co_sa)

# Transform

In [13]:
translator = GoogleTranslator(source='auto', target='es')

country_map = {}
for country in df_geo['country_name'].unique():
    try:
        translated = translator.translate(country)
    except Exception:
        translated = country
    country_map[country] = translated

df_geo['spanish_country_region_name'] = df_geo['country_name'].map(country_map)

print(country_map)

{'United States': 'Estados Unidos', 'Canada': 'Canadá', 'France': 'Francia', 'Germany': 'Alemania', 'Australia': 'Australia', 'United Kingdom': 'Reino Unido'}


# Load

In [16]:
df_geo.to_sql(
    'dim_geography',
    etl_conn,
    schema='dw',
    if_exists='replace',  # usa 'append' en producción
    index=False
)

614